# 1. Zapewne pamiętasz o odkurzaczu omówionym w pierwszych dwóch wykładach.
(i) Sformułuj problem jako problem przeszukiwania; tzn. musisz napisać stan
początkowy, możliwe działania za pomocą funkcji Action(state), relację
przejścia za pomocą funkcji Result(state, action) i stan celu. Proszę zobacz
yć strona 9 wykładu 2. Zakładamy, że obie kwadraty A i B są brudne i
agent znajduje się w kwadracie A. Zakładamy również, że agent ma trzy

możliwe działania: ruszaj w lewo, ruszaj w prawo i ssać. (ii) Napisz pro-
gram za pomocą algorytm ‘Breadth First Search, aby agent mógł odszukiwać

brud we wszystkich możliwych lokalizacjach i wyczyścić go (tzn., znalezienie
ścieżkę z węzła początkowego do węzła docelowego. Wystarczy, aby program
wydrukował każdą sekwencję akcji na konsoli, i pokazał kiedy osiąga cel
(czyli ścieżka z węzła początkowego do węzła docelowego).

In [1]:
from collections import deque

def find_shortest_cleaning_sequence(initial_state):
    # Initialize the queue with starting state and empty path
    queue = deque()
    queue.append((initial_state, []))
    
    # Track visited states to avoid cycles
    visited = set()
    
    while queue:
        current_state, path = queue.popleft()
        
        # Create a unique key for the state to check if we've seen it before
        state_key = (
            tuple(sorted(current_state['cleanliness'].items())),
            current_state['vacuum']
        )
        
        # Skip if we've already seen this state
        if state_key in visited:
            continue
            
        # Check if we've reached the goal (all positions clean)
        if all(status == 'clean' for status in current_state['cleanliness'].values()):
            return path
            
        # Mark this state as visited
        visited.add(state_key)
        
        # Generate all possible next states
        current_pos = current_state['vacuum']
        positions = current_state['positions']
        
        # Action 1: Clean current position
        if current_state['cleanliness'][current_pos] == 'dirty':
            new_state = current_state.copy()
            new_state['cleanliness'] = current_state['cleanliness'].copy()
            new_state['cleanliness'][current_pos] = 'clean'
            queue.append((new_state, path + ['clean']))
        
        # Action 2: Move left (to previous position in the list)
        current_index = positions.index(current_pos)
        if current_index > 0:
            new_pos = positions[current_index - 1]
            new_state = current_state.copy()
            new_state['vacuum'] = new_pos
            queue.append((new_state, path + ['move_left']))
        
        # Action 3: Move right (to next position in the list)
        if current_index < len(positions) - 1:
            new_pos = positions[current_index + 1]
            new_state = current_state.copy()
            new_state['vacuum'] = new_pos
            queue.append((new_state, path + ['move_right']))
    
    return None 

In [3]:
initial_state = {
    'positions': ['A', 'B', 'C'],  # Can add more positions like ['A', 'B', 'C']
    'cleanliness': {'A': 'dirty', 'B': 'clean', 'C': 'dirty'},
    'vacuum': 'B'
}

solution = find_shortest_cleaning_sequence(initial_state)
print("Shortest cleaning sequence:", solution)

Shortest cleaning sequence: ['move_left', 'clean', 'move_right', 'move_right', 'clean']


# 2. Rozważ 8 zagadek ze stanem początkowym jako
[ ] [1] [3]
[4] [2] [5]
[7] [8] [6].

Stan docelowy to:
[1] [2] [3]
[4] [5] [6]
[7] [8] [ ].

(i) Opisz sformułowanie problemu jako problem przeszukiwania.
(ii) Napisz program, używając przeszukiwania A∗

(Zobacz numer wykładu 3). Uważamy, że każdy krok kosztuje 1, a zatem koszty osiągnięcia stanu n 
z stanu początkowego (czyli g(n)) jest całkowita liczba kroków wymaganych do osiągnięcia stanu n. Możesz użyć funkcji heurystycznej, gdzie h(stan) = suma odległości bloku miasta (sum of city block distance). Wystarczy, aby 
program wydrukował każdą sekwencję akcji na konsoli, i pokazał kiedy osiąga cel.

(Uwaga: W celu wykonania (ii) byłoby dobrze stworzyć drzewo gry z powiązaną wartością f(n) = g(n) + h(n) dla każdego węzła.)

In [18]:
import numpy as np
import heapq

class Puzzle8:
    def __init__(self, start, goal):
        self.start = tuple(map(tuple, start))
        self.goal = tuple(map(tuple, goal))
        self.n = len(start)
        self.moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    def heuristic(self, state):
        state_array = np.array(state)
        goal_array = np.array(self.goal)
        h = sum(abs(x1 - x2) + abs(y1 - y2)
                for i in range(1, 9)
                for (x1, y1), (x2, y2) in zip(np.argwhere(state_array == i), np.argwhere(goal_array == i)))
        return h

    def get_neighbors(self, state):
        neighbors = []
        state_array = np.array(state)
        x, y = np.argwhere(state_array == None)[0]

        for dx, dy in self.moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.n and 0 <= ny < self.n:
                new_state = state_array.copy()
                new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x, y]
                neighbors.append(tuple(map(tuple, new_state)))
        return neighbors

    def solve(self):
        open_set = []
        heapq.heappush(open_set, (self.heuristic(self.start), 0, self.start, []))
        visited = set()

        while open_set:
            _, g, current, path = heapq.heappop(open_set)

            if current in visited:
                continue
            visited.add(current)

            if current == self.goal:
                print(f"Znaleziono rozwiązanie w {g} ruchach!")
                for step in path:
                    print(np.array(step), "\n")
                return path

            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    new_g = g + 1
                    heapq.heappush(open_set, (new_g + self.heuristic(neighbor), new_g, neighbor, path + [neighbor]))

        print("Brak rozwiązania")
        return None

start_state = np.array([[None, 1, 3],
                        [4, 2, 5],
                        [7, 8, 6]])

goal_state = np.array([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, None]])

puzzle = Puzzle8(start_state, goal_state)
solution = puzzle.solve()


Znaleziono rozwiązanie w 4 ruchach!
[[1 None 3]
 [4 2 5]
 [7 8 6]] 

[[1 2 3]
 [4 None 5]
 [7 8 6]] 

[[1 2 3]
 [4 5 None]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 None]] 


3. Zapewne pamiętasz o Trywialnej grze rozważanej w wykładach 3 i 4. Napisz
program, aby znaleźć optymalną strategię dla MAX za pomocą algorytmu
Maximin z przycinaniem [α, β] ([α, β]-prunning).